In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers
from os import listdir

In [2]:
data_list_model2c = listdir(r"C:\Users\gowri\OneDrive\Desktop\covidDetection\train")
data_list_model2c


['bacterial_viral', 'COVID']

In [3]:
DATASET_PATH_model2c  = r"C:\Users\gowri\OneDrive\Desktop\covidDetection\train"
train_dir_model2c =  r"C:\Users\gowri\OneDrive\Desktop\covidDetection\train"
test_dir_model2c =  r"C:\Users\gowri\OneDrive\Desktop\covidDetection\test"

In [4]:
IMAGE_SIZE    = (150, 150)
NUM_CLASSES   = len(data_list_model2c)
BATCH_SIZE    = 10  # try reducing batch size or freeze more layers if your GPU runs out of memory
NUM_EPOCHS    = 100
LEARNING_RATE = 0.0001

In [5]:
#Train datagen here is a preprocessor
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=50,
                                   featurewise_center = True,
                                   featurewise_std_normalization = True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.25,
                                   zoom_range=0.1,
                                   zca_whitening = True,
                                   channel_shift_range = 20,
                                   horizontal_flip = True ,
                                   vertical_flip = True ,
                                   fill_mode='constant' )

c:\Users\gowri\anaconda3\Lib\site-packages\keras\src\legacy\preprocessing\image.py:1054: UserWarning: This ImageDataGenerator specifies `zca_whitening` which overrides setting of`featurewise_std_normalization`.
  warnings.warn(


In [6]:
train_batches = train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "training",
                                                  seed=42,
                                                  class_mode="categorical",)

Found 175 images belonging to 2 classes.


In [7]:
valid_batches = train_datagen.flow_from_directory(DATASET_PATH_model2c,
                                                  target_size=IMAGE_SIZE,
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE,
                                                  subset = "validation",
                                                  seed=42,
                                                  class_mode="categorical",)

Found 0 images belonging to 2 classes.


In [8]:
vgg16_model = tf.keras.applications.vgg16.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(150,150,3)))

In [9]:
for layer in vgg16_model.layers[:-4]:
    layer.trainable = False
model2c = tf.keras.Sequential()
model2c.add(vgg16_model)
model2c.add(Flatten())
model2c.add(Dense(64, activation='relu'))
model2c.add(layers.Dense(2, activation='softmax'))

In [11]:
model2c.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])

In [12]:
STEP_SIZE_TRAIN=train_batches.n//train_batches.batch_size
STEP_SIZE_VALID=valid_batches.n//valid_batches.batch_size

In [14]:
model2c.fit(train_batches, steps_per_epoch =STEP_SIZE_TRAIN, epochs= 5, verbose=True)

c:\Users\gowri\anaconda3\Lib\site-packages\keras\src\legacy\preprocessing\image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
c:\Users\gowri\anaconda3\Lib\site-packages\keras\src\legacy\preprocessing\image.py:1286: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/5


c:\Users\gowri\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


17/17 ━━━━━━━━━━━━━━━━━━━━ 21s 821ms/step - accuracy: 0.6274 - loss: 0.6872
Epoch 2/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9000 - loss: 0.4374   
Epoch 3/5


c:\Users\gowri\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


17/17 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.8205 - loss: 0.4136
Epoch 4/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7000 - loss: 0.7506
Epoch 5/5
17/17 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.8318 - loss: 0.3358


In [18]:
Train_DataGeneration = ImageDataGenerator(rescale=1.0 / 255)
TrainBatch = Train_DataGeneration.flow_from_directory(
    train_dir_model2c, 
    target_size=IMAGE_SIZE, 
    batch_size=1, 
    shuffle=False, 
    seed=42, 
    class_mode="categorical"
)

TrainBatch.reset()

# Evaluating the model (Fixed)
trainResult = model2c.evaluate(TrainBatch, steps=len(TrainBatch), verbose=1)
print('Train loss:', trainResult[0], 'Train accuracy:', trainResult[1])

# Making predictions
prediction = model2c.predict(TrainBatch, steps=len(TrainBatch))
Predicted_class = np.argmax(prediction, axis=1)

# Generating confusion matrix
print(confusion_matrix(TrainBatch.classes, Predicted_class))


Found 175 images belonging to 2 classes.


  2/175 ━━━━━━━━━━━━━━━━━━━━ 13s 81ms/step - accuracy: 1.0000 - loss: 0.0381  

c:\Users\gowri\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


175/175 ━━━━━━━━━━━━━━━━━━━━ 32s 179ms/step - accuracy: 0.9584 - loss: 0.1219
Train loss: 0.11304084956645966 Train accuracy: 0.9599999785423279
175/175 ━━━━━━━━━━━━━━━━━━━━ 28s 158ms/step
[[72  3]
 [ 4 96]]


In [22]:
# Test accuracy
Test_DataGeneration = ImageDataGenerator(rescale=1.0 / 255)
TestingBatch = Test_DataGeneration.flow_from_directory(
    test_dir_model2c, 
    target_size=IMAGE_SIZE, 
    batch_size=1, 
    shuffle=False, 
    seed=42, 
    class_mode="categorical"
)

TestingBatch.reset()

# Evaluating the model (Fixed)
testResult = model2c.evaluate(TestingBatch, steps=len(TestingBatch), verbose=1)
print('Test loss:', testResult[0], 'Test accuracy:', testResult[1])

# Making predictions (Fixed)
prediction = model2c.predict(TestingBatch, steps=len(TestingBatch))
Predicted_class = np.argmax(prediction, axis=1)

# Generating confusion matrix
print(confusion_matrix(TestingBatch.classes, Predicted_class))


Found 50 images belonging to 2 classes.
 1/50 ━━━━━━━━━━━━━━━━━━━━ 9s 185ms/step - accuracy: 1.0000 - loss: 0.1708

c:\Users\gowri\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 83ms/step - accuracy: 0.9315 - loss: 0.1661
Test loss: 0.10606435686349869 Test accuracy: 0.9599999785423279
50/50 ━━━━━━━━━━━━━━━━━━━━ 4s 89ms/step
[[23  2]
 [ 0 25]]
